<span style="color:darkviolet">
Legal-BERT: zlucia/legalbert (https://huggingface.co/zlucia/legalbert)
</span>

<span style="color:darkviolet">
1) 50 files <br>
2) 8 different categories of unfair clauses (xml tag for each clause) <br>
3) clearly fair (1), potentially unfair (2) , or clearly unfair (3). <br>
4)  12,011 sentences <br>
</span>

In [ ]:
# !pip install -r requirements.txt
# !pip install transformers

In [ ]:
# !pip install git+https://github.com/huggingface/transformers

In [1]:
import os
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [2]:
files = os.listdir("./data/ToS/Sentences/")

In [3]:
len(files)

50

In [ ]:
files

<span style="color:darkviolet">
Prepare the Training Data (Combine sentences and Labels)
</span>

In [4]:
prefix_sentence = "./data/ToS/Sentences/"
prefix_label = "./data/ToS/Labels/"

In [5]:
clauses = []
for file in files:
    sentence_file_path = prefix_sentence + file 
    label_file_path = prefix_label + file
    sentences_df = pd.read_csv(sentence_file_path, sep="dummy_separator", header=None)
    sentences_df.columns = ["sentences"]
    label_df = pd.read_csv(label_file_path, sep=" ", header=None)
    label_df.columns = ["label"]
    label_df["label_converted"] = np.where(label_df["label"] == -1, 0, 1)
    df_concat = pd.concat([label_df["label_converted"], sentences_df], axis=1)
    clauses.append(df_concat)

In [6]:
colnames = ["sentences", "label_converted"]
clauses_df = pd.DataFrame(columns = colnames)
for df in clauses:
    clauses_df = clauses_df.append(df)

In [ ]:
clauses_df

In [7]:
clauses_df.rename(columns={'label_converted': 'label', 'sentences': 'sentences'}, inplace=True)

In [ ]:
clauses_df

In [ ]:
assert (clauses_df.isnull().sum().all() == 0)

In [8]:
clauses_df.to_csv("./data/tos_clauses.csv", index=False)

<span style="color:darkviolet">
Compute pretrain loss
</span>

In [9]:
%env TOKENIZERS_PARALLELISM=false

env: TOKENIZERS_PARALLELISM=false


In [10]:
# Download model from Hugging Face model repository
!python3 classification/run_glue.py \
  --model_name_or_path zlucia/legalbert \
  --train_file data/tos_clauses.csv \
  --validation_file data/tos_clauses.csv \
  --ptl=True \
  --max_seq_length 128 \
  --output_dir logs/ToS/legalbert \
  --overwrite_output_dir

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
Moving 0 files to the new cache system
0it [00:00, ?it/s]
10/23/2022 13:57:36 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
10/23/2022 13:57:36 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluatio

[INFO|configuration_utils.py:653] 2022-10-23 13:57:38,046 >> loading configuration file config.json from cache at /Users/akankshanogaja/.cache/huggingface/hub/models--zlucia--legalbert/snapshots/8aead03573cac81f97191bd8532d808889372e59/config.json
[INFO|configuration_utils.py:705] 2022-10-23 13:57:38,047 >> Model config BertConfig {
  "_name_or_path": "zlucia/legalbert",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absol

<span style="color:darkviolet">
Step 3: Split data into train and dev. Fine tune on train dataset and evaluate performance on dev dataset. Logs and Predictions are directed to output directory logs/ToS/legalbert
</span>

In [11]:
train, dev = train_test_split(clauses_df, test_size=0.2, random_state=42, stratify=clauses_df[['label']])

In [12]:
train.to_csv("./data/tos_clauses_train.csv", index=False)
dev.to_csv("./data/tos_clauses_dev.csv", index=False)

In [13]:
# Download model from Hugging Face model repository
!python3 classification/run_glue.py \
  --model_name_or_path zlucia/legalbert \
  --train_file data/tos_clauses_train.csv \
  --validation_file data/tos_clauses_dev.csv \
  --do_train \
  --do_eval \
  --evaluation_strategy steps \
  --max_seq_length 128 \
  --per_device_train_batch_size=16 \
  --learning_rate=1e-5 \
  --num_train_epochs=2.0 \
  --output_dir logs/ToS/legalbert \
  --overwrite_output_dir \
  --logging_steps 50

10/23/2022 15:01:04 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False
10/23/2022 15:01:04 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=50,
evaluation_strategy=steps,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,

[INFO|configuration_utils.py:653] 2022-10-23 15:01:06,861 >> loading configuration file config.json from cache at /Users/akankshanogaja/.cache/huggingface/hub/models--zlucia--legalbert/snapshots/8aead03573cac81f97191bd8532d808889372e59/config.json
[INFO|configuration_utils.py:705] 2022-10-23 15:01:06,861 >> Model config BertConfig {
  "_name_or_path": "zlucia/legalbert",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absol

{'loss': 0.4802, 'learning_rate': 9.469214437367304e-06, 'epoch': 0.11}         
  5%|██▏                                       | 50/942 [01:42<31:28,  2.12s/it][INFO|trainer.py:725] 2022-10-23 15:02:51,679 >> The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentences. If sentences are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:2907] 2022-10-23 15:02:51,681 >> ***** Running Evaluation *****
[INFO|trainer.py:2909] 2022-10-23 15:02:51,681 >>   Num examples = 1883
[INFO|trainer.py:2912] 2022-10-23 15:02:51,681 >>   Batch size = 8

 38%|███████████████▊                          | 89/236 [00:30<00:50,  2.94it/s]


 79%|████████████████████████████████▎        | 186/236 [01:02<00:16,  2.95it/s]


100%|█████████████████████████████████████████| 236/236 [01:19<00:00,  3.49it/s]/Users/akankshanogaja/Downloads/CSCI_544_ANLP/Project/classification/run_glue.py:305: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("f1")



{'eval_loss': 0.3246283531188965, 'eval_f1': 0.0, 'eval_runtime': 80.9104, 'eval_samples_per_second': 23.273, 'eval_steps_per_second': 2.917, 'epoch': 0.11}

  5%|██▏                                       | 50/942 [03:03<31:28,  2.12s/it]
{'loss': 0.3178, 'learning_rate': 8.938428874734608e-06, 'epoch': 0.21}         
 11%|████▎                                    | 100/942 [04:47<28:46,  2.05s/it][INFO|trainer.py:725] 2022-10-23 15:05:55,918 >> The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence

...

 82%|█████████████████████████████████▋       | 194/236 [01:02<00:13,  3.16it/s]


100%|████████████████████████████████████████▊| 235/236 [01:15<00:00,  3.00it/s]
                                                                                
{'eval_loss': 0.18189361691474915, 'eval_f1': 0.7469287469287469, 'eval_runtime': 76.9884, 'eval_samples_per_second': 24.458, 'eval_steps_per_second': 3.065, 'epoch': 1.8}
100%|█████████████████████████████████████████| 236/236 [01:16<00:00,  3.61it/s]
{'loss': 0.1503, 'learning_rate': 4.45859872611465e-07, 'epoch': 1.91}          
 96%|███████████████████████████████████████▏ | 900/942 [54:19<01:27,  2.08s/it][INFO|trainer.py:725] 2022-10-23 15:55:28,484 >> The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentences. If sentences are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
[INFO|trainer.py:2907] 2022-10-23 15:55:28,486 >> ***** Running Evaluation *****
[INFO|trainer.py:2909] 202

 59%|████████████████████████▎                | 140/236 [00:44<00:30,  3.14it/s]


100%|████████████████████████████████████████▊| 235/236 [01:14<00:00,  3.16it/s]
                                                                                
{'eval_loss': 0.17827272415161133, 'eval_f1': 0.7532467532467533, 'eval_runtime': 75.9949, 'eval_samples_per_second': 24.778, 'eval_steps_per_second': 3.105, 'epoch': 1.91}
100%|█████████████████████████████████████████| 236/236 [01:15<00:00,  3.76it/s]


100%|█████████████████████████████████████████| 942/942 [57:04<00:00,  1.93s/it][INFO|trainer.py:1852] 2022-10-23 15:58:13,229 >> 

Training completed. Do not forget to share your model on huggingface.co/models =)


{'train_runtime': 3424.4608, 'train_samples_per_second': 4.398, 'train_steps_per_second': 0.275, 'train_loss': 0.18288464809425825, 'epoch': 2.0}
100%|█████████████████████████████████████████| 942/942 [57:04<00:00,  3.64s/it]
[INFO|trainer.py:2656] 2022-10-23 15:58:13,230 >> Saving model checkpoint to logs/ToS/legalbert
[INFO|configuration_utils.py:447] 2022-10-23 15:58:13,232 >> Configuration saved in logs/ToS/legalbert/config.json
[INFO|modeling_utils.py:1624] 2022-10-23 15:58:13,443 >> Model weights saved in logs/ToS/legalbert/pytorch_model.bin
[INFO|tokenization_utils_base.py:2123] 2022-10-23 15:58:13,444 >> tokenizer config file saved in logs/ToS/legalbert/tokenizer_config.json
[INFO|tokenization_utils_base.py:2130] 2022-10-23 15:58:13,444 >> Special tokens file saved